# Data types
1. Rational
2. Complex
3. High-precision

# Linear algebra
1. Solving linear systems/left divide
2. Special matrices (tridiagonal, identity, etc)

## Remembering how things work

In [1]:
A = [1 2; 3 4]
B = [5 7; 1 -3]

2×2 Matrix{Int64}:
 5   7
 1  -3

In [2]:
typeof(1/3)

Float64

In [3]:
X = A \ B   # casts to Floats

2×2 Matrix{Float64}:
 -9.0  -17.0
  7.0   12.0

In [4]:
A * X == B

false

In [5]:
A * X ≈ B

true

In [6]:
Br = Rational.(B)
Ar = Rational.(A)
Xr = Ar \ Br

2×2 Matrix{Rational{Int64}}:
 -9//1  -17//1
  7//1   12//1

In [7]:
Ar * Xr == Br

true

# Exercises
<!-- exercise: implement Gram-Schmidt in order to perform QR factorization on a matrix
exercise: something like solving the heat equation or something else with a tridiagonal matrix
exercise: something with high orders of convergence that requires bigs
exercise: implement partial fraction decomposition
exercise: angle identities using complex?
exercise: contour integrals? -->
## Integrating rational functions
Throughout this exercise, we will be working with polynomials and so we have to settle on a good representation. Since polynomials can always be factorized over $\mathbb{C}$, we will identify polyonmials by their __factors__, i.e. $p(x) = (x-a_1)(x-a_2)\ldots(x-a_k)$ is represented by the tuple 
`
p = (a_1, a_2, ..., a_k)
`.

### Part 1
Implement [polynomial long division](https://en.wikipedia.org/wiki/Polynomial_long_division). Specifically, given two polynomials $p(x)$ and $q(x)$, we can write 
$$p(x) = q(x) * d(x) + r(x)$$
where $\deg(r)<\deg(q)$. 

1. Your code should accept two polynomials, $p$ and $q$, and compute the polynomials $d$ and $r$.
2. _Hint_: can you set up a linear algebra problem to solve this?

### Part 2
Implement polynomial integration. Your code should accept a polynomial $p$ as well as left and right endpoints $x_0$ and $x_1$ and it should compute 
$$\int_{x_0}^{x_1} p(x) \mathrm{d} x$$ 
exactly.

### Part 3
Next, implement [partial fraction decomposition](https://en.wikipedia.org/wiki/Partial_fraction_decomposition) using _complex rational_ numbers. Specifically, given two polynomials $p(x)$ and $q(x)$ with $\deg(p)<\deg(q)$, we can write 
$$ \frac{p(x)}{q(x)} = \frac{c_1}{(x-a_1)}+\ldots +\frac{c_2}{(x-a_2)}.$$

1. Your code should accept two polynomials, $p$ and $q$, and compute the coefficients $c_i$.
2. _Hint_: start with the case where all factors of $q$ are distinct, then figure out the case where there are repeated factors.
3. _Hint_: can you set up a linear algebra problem to solve this?

### Part 4
Implement rational integration. Your code should accept a simple rational a coefficient $c$ and a root $a$ as well as left and right endpoints $x_0$ and $x_1$ and it should compute 
$$\int_{x_0}^{x_1} \frac{c}{x-a} \mathrm{d} x$$
exactly.

1. There may be problems when $a$ is real and $x_0<a<x_1$. You can start ignore this case. If you want to address it, try implementing the Cauchy principal value.

### Part 5
Combine polynomial long division and partial fraction decomposition to simplify any rational function for integration. Specifically, given any polynomials $p$ and $q$, find polynomial $e$ and coefficients $c_i$ such that 
$$\frac{p(x)}{q(x)} = e(x) + \frac{c_1}{(x-a_1)} +\ldots +\frac{c_k}{(x-a_k)}$$
where $a_i$ are the roots of $q$.

### Part 6: 
Put parts 1-5 together to compute any integral of the form 
$$\int_{x_0}^{x_1} \frac{p(x)}{q(x)}\mathrm{d} x.$$

1. Modify your code so that it supports arbitrary precision computing.

### Part 7
Test your code.

1. Plot the integral of $$\frac{1}{1+x^2}$$ for $x_0=0$ and multiple values of $x_1$. What does the curve look like? Is this what you expected?
2. Compute $\pi$ to 25 digits using the above integral and two endpoints of your choice.
3. How good are [Pade approximants](https://en.wikipedia.org/wiki/Padé_approximant)? Pade approximants are like Taylor series, but using rational functions instead of polynomials. The degree (2,2) Pade approximation of $e^x$ is $$f(x) = \frac{1+\frac{x}{2}+\frac{x^2}{9}}{1-\frac{x}{2}+\frac{x^2}{9}}.$$ Assess the quality of the approximation by plotting (1) $f(x)$ and (2) $1 + \int_0^x f(y)\mathrm{d} y$. 

## Numerical integration, convergence rates, arbitrary precision

The standard strategy for numerical integration is to use a quadrature rule. You already learned some quadrature rules when you learned about Riemann integration:

1. Left endpoint: we approximate $$\int_a^b f(x)\mathrm{d} x \approx I_h(f)= h\sum_{j=0}^{N-1}f(x_j)$$ where $x_j=a+hj$ and $h=\frac{b-a}{N}$.
2. Midpoint: $$\int_a^b f(x)\mathrm{d} x \approx I_h(f)=h\sum_{j=0}^{N-1}f(x_{j+1/2})$$ where $x_{j+1/2} = a +hj + \frac{h}{2}$.

### Part 1
Code up the left endpoint and midpoint quadrature schemes. Your code should have the signatures
```julia
function left_endpoint_quadrature(f, a, b, N)
    # code here
end
```
and 
```julia
function midpoint_quadrature(f, a, b, N)
    # code here
end
```

### Part 2 
Test your code by computing the integral of $\sin^2$ on $[0,2\pi]$. Do this for multiple values of $N$, and create a log-log plot of error of the integral vs $N$.

### Part 3
Here is a high-order quadrature rule to approximate an integral on $[0,1]$:
$$\int_{-1}^1 f(x)\mathrm{d} x = \sum w_if(x_i)$$
where $(x_i, w_i) \in\left\{(0,32/45), (\pm\sqrt{3/7}, 49/90), (\pm 1, 1/10)\right\}$. Use this rule to compute the integrals on each interval $[jh, (j+1)h]$ as before with the left endpoint and midpoint methods.